In [1]:
import torch
import sys
import numpy as np
import itertools
import models
from models import ConvLSTM
from dataset import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
import argparse
import time
import datetime
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.environ['CUDA_VISIBLE_DEVICES'] = '2'


In [2]:
from flickerfusion.utils import *
from flickerfusion.attacks import LinfAttack, L2Attack

In [3]:
model_path = 'model_checkpoints/model_nodecay_30.pth'

In [4]:
device = "cuda"

In [5]:
!nvidia-smi

Sun Jun  4 12:48:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            Off  | 00000000:05:00.0 Off |                  N/A |
| 20%   41C    P0    61W / 250W |      0MiB / 12194MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN Xp            Off  | 00000000:09:00.0 Off |                  N/A |
| 33%   50C    P0    60W / 250W |      0MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [6]:
# Define network
model = ConvLSTM(
    num_classes=101,
    latent_dim=512,
    lstm_layers=1,
    hidden_dim=1024,
    bidirectional=True,
    #attention=True,
)
model = model.to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

ConvLSTM(
  (encoder): Encoder(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)

In [7]:
# Get videos with all frames, as a result, batch_size should be 1
test_dataset = Dataset(
    dataset_path="data/UCF-101-frames/",
    split_path="data/ucfTrainTestlist",
    split_number=1,
    input_shape=(3,224,224),
    sequence_length=40,
    training=False,
)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

/home/sangwooji/miniconda3/envs/colorfusion/lib/python3.7/site-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [8]:
correct = total = 0
for batch_i, (images, labels) in enumerate(test_dataloader):
    break
images = images.to(device)
labels = labels.to(device)

In [9]:
net = model

In [10]:
perturbation = torch.ones(3,224,224).to(device)
perturbation.requires_grad = True

In [11]:
# net.lstm.train()
torch.backends.cudnn.enabled = False

In [12]:
net.eval()
adv = images+perturbation
adv_result = net(adv)

In [13]:
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(adv_result, labels)

In [14]:
loss.backward()

In [15]:
perturbation.grad

tensor([[[-4.8964e-05,  1.3020e-05,  5.3427e-05,  ..., -2.7034e-06,
           3.1119e-05,  3.4413e-05],
         [-7.7601e-05,  8.9612e-05,  1.2403e-04,  ...,  1.0433e-05,
           3.0507e-05,  2.9955e-05],
         [-5.4946e-05,  2.3295e-05,  1.3957e-04,  ..., -1.1914e-05,
          -3.7560e-06,  3.0114e-05],
         ...,
         [-2.7529e-05, -2.8413e-05,  1.7740e-05,  ...,  5.0725e-05,
           2.5488e-05,  8.6416e-06],
         [-1.6701e-05, -1.5975e-06, -9.9887e-08,  ..., -1.6522e-05,
           2.2767e-05, -1.3772e-06],
         [ 1.7332e-05,  7.3597e-06, -4.6168e-06,  ...,  2.3592e-05,
          -4.2898e-06, -1.1193e-04]],

        [[-3.2781e-05, -1.6968e-06,  2.1020e-05,  ..., -3.6889e-05,
          -8.0709e-07,  1.5357e-05],
         [-6.4874e-05,  6.6567e-05,  6.5496e-05,  ..., -3.6593e-05,
          -1.0825e-05, -1.4934e-05],
         [-5.5583e-05, -7.7941e-06,  1.0270e-04,  ..., -3.7225e-05,
          -3.9511e-05, -2.4441e-05],
         ...,
         [-4.0342e-05, -2

In [16]:
!nvidia-smi

Sun Jun  4 12:49:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            Off  | 00000000:05:00.0 Off |                  N/A |
| 21%   41C    P0    61W / 250W |      0MiB / 12194MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN Xp            Off  | 00000000:09:00.0 Off |                  N/A |
| 29%   54C    P2    79W / 250W |   9239MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [12]:
perturbation = torch.ones(3,224,224).to(device)
perturbation.requires_grad = True
adv = images+perturbation
torch.backends.cudnn.enabled = False

In [13]:
model = models.Encoder(
    latent_dim=512,
)
model.eval()
model = model.to(device)

In [29]:
results = model(adv[0])

In [30]:
results.shape

torch.Size([164, 512])

In [31]:
toLogit = torch.nn.Linear(164*512, 101).to(device)
logit = toLogit(results.reshape(1,-1))
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(logit, labels)
loss.backward()

In [32]:
print(perturbation.grad)

None


In [68]:
lstm = models.LSTM(512, 1, 1024, True)
lstm.to(device)

LSTM(
  (lstm): LSTM(512, 1024, batch_first=True, bidirectional=True)
)

In [69]:
results2 = lstm(results.view(1, results.shape[0], -1))

In [70]:
results2.shape

torch.Size([1, 164, 2048])

In [71]:
linear = torch.nn.Linear(2048, 1)
linear.to(device)

Linear(in_features=2048, out_features=1, bias=True)

In [73]:
results3 = linear(results2)

In [78]:
import torch.nn.functional as F

In [80]:
results4 = F.softmax(results3.squeeze(-1), dim=-1)

In [89]:
results5 = torch.sum(results4.unsqueeze(-1) * results2, dim=1)

In [92]:
results5.grad_fn

In [99]:
hidden_dim = 1024 
num_classes = 101
output_layers = torch.nn.Sequential(
    torch.nn.Linear(2 * hidden_dim, hidden_dim),
    torch.nn.BatchNorm1d(hidden_dim, momentum=0.01),
    torch.nn.ReLU(),
    torch.nn.Linear(hidden_dim, num_classes),
    torch.nn.Softmax(dim=-1),
)
output_layers.eval()
output_layers.to(device)

Sequential(
  (0): Linear(in_features=2048, out_features=1024, bias=True)
  (1): BatchNorm1d(1024, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Linear(in_features=1024, out_features=101, bias=True)
  (4): Softmax(dim=-1)
)

In [100]:
results5.shape

torch.Size([1, 2048])

In [101]:
results6 = output_layers(results5)

In [102]:
results6

tensor([[0.0101, 0.0096, 0.0101, 0.0096, 0.0101, 0.0100, 0.0098, 0.0102, 0.0097,
         0.0099, 0.0099, 0.0101, 0.0099, 0.0098, 0.0098, 0.0102, 0.0103, 0.0097,
         0.0101, 0.0101, 0.0102, 0.0102, 0.0100, 0.0097, 0.0101, 0.0098, 0.0102,
         0.0101, 0.0098, 0.0099, 0.0100, 0.0100, 0.0099, 0.0102, 0.0100, 0.0096,
         0.0099, 0.0097, 0.0098, 0.0098, 0.0097, 0.0102, 0.0098, 0.0099, 0.0098,
         0.0098, 0.0097, 0.0098, 0.0101, 0.0098, 0.0098, 0.0103, 0.0102, 0.0102,
         0.0099, 0.0099, 0.0096, 0.0098, 0.0100, 0.0099, 0.0097, 0.0096, 0.0097,
         0.0098, 0.0100, 0.0098, 0.0097, 0.0102, 0.0101, 0.0097, 0.0096, 0.0100,
         0.0101, 0.0098, 0.0097, 0.0100, 0.0097, 0.0099, 0.0098, 0.0097, 0.0098,
         0.0098, 0.0099, 0.0099, 0.0098, 0.0098, 0.0099, 0.0098, 0.0100, 0.0098,
         0.0098, 0.0100, 0.0098, 0.0101, 0.0099, 0.0102, 0.0097, 0.0099, 0.0099,
         0.0100, 0.0097]], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [103]:
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(results6, labels)

In [112]:
loss.backward()

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling .backward() or autograd.grad() the first time.

In [113]:
perturbation.grad

True

In [13]:
net = torch.nn.LSTM(512, 1024, 2)

In [14]:
net.eval()

LSTM(512, 1024, num_layers=2)

In [15]:
sample = torch.rand(1,1,512)

In [16]:
perturbation = torch.ones_like(sample)
perturbation.requires_grad = True

In [17]:
adv = sample + perturbation

In [18]:
result = net(adv)[0]

In [19]:
linear = torch.nn.Linear(1024, 10)

In [20]:
r = linear(result)

In [21]:
loss_fn = torch.nn.CrossEntropyLoss()

In [22]:
label = torch.ones(1).long()

In [23]:
loss = loss_fn(r[0], label)

In [24]:
loss.backward()

In [25]:
perturbation.grad.shape

torch.Size([1, 1, 512])

In [26]:
perturbation.grad

tensor([[[-1.9206e-04, -4.5834e-04, -7.0031e-05,  2.0036e-04,  3.6439e-04,
           5.9452e-06,  6.6198e-04,  5.8982e-04,  3.6445e-04,  1.2748e-04,
           1.4688e-04, -1.4086e-04,  4.8118e-04,  5.5115e-05,  5.0407e-04,
           5.0873e-04, -1.1349e-04, -1.1487e-04,  7.0009e-05, -2.5364e-04,
           7.5248e-05,  1.5622e-04, -8.0532e-04, -2.1022e-04, -1.6068e-05,
          -2.3742e-04, -5.0388e-05, -3.3451e-04,  1.1708e-05, -5.0915e-04,
           4.9320e-05, -6.1708e-04,  8.3335e-05,  2.9895e-04, -4.1996e-04,
          -4.3888e-04,  3.3830e-04, -1.2647e-04,  5.7912e-04, -1.5746e-04,
           2.3566e-04,  1.1325e-04, -2.6284e-04,  4.6794e-04,  1.5082e-04,
           1.4786e-04,  1.4837e-04,  2.1347e-05,  6.4414e-04, -3.8998e-05,
          -3.1261e-05,  4.9590e-04, -1.8191e-04,  1.8967e-04,  1.8622e-04,
          -3.1657e-05,  1.9699e-04,  3.6595e-04,  1.5152e-04,  2.6320e-04,
          -2.0029e-05, -9.6789e-05, -8.2520e-05, -6.9029e-04,  1.2553e-04,
          -2.9264e-04,  6

In [18]:
from torchvision.models.resnet import resnet50

In [28]:
resnet = resnet50().to(device)

In [29]:
result = resnet(adv[0,0:1])

In [31]:
loss = loss_fn(result, labels)

In [32]:
loss.backward()

In [33]:
perturbation.grad

tensor([[[-1.1813e-03,  5.8156e-03, -1.7487e-03,  ...,  2.2420e-05,
          -4.3434e-03,  1.6723e-03],
         [-1.9304e-02,  4.5507e-03, -1.9298e-03,  ...,  6.3596e-04,
           2.0108e-03, -9.2278e-03],
         [-6.9206e-03,  1.1881e-02,  3.8085e-03,  ..., -9.1914e-03,
           2.1022e-03,  4.6818e-04],
         ...,
         [ 1.4537e-02, -8.6498e-03, -3.7245e-03,  ..., -1.3832e-03,
          -2.8592e-03, -7.7448e-04],
         [ 4.9330e-03,  3.1738e-03,  4.4205e-03,  ..., -4.3441e-03,
           1.9800e-03, -9.4934e-04],
         [ 3.4057e-03,  3.8184e-03,  4.7175e-03,  ...,  3.6633e-03,
          -9.1093e-04,  1.0815e-03]],

        [[ 1.0627e-02,  3.3389e-03,  1.3233e-03,  ...,  2.2948e-03,
           7.0601e-03,  1.8375e-05],
         [-2.3413e-03,  7.8030e-03,  1.9006e-02,  ..., -3.1056e-03,
           7.1637e-03, -4.7506e-03],
         [-1.9420e-03, -1.1209e-02, -5.3635e-04,  ..., -9.8420e-04,
           5.6251e-03,  8.0061e-03],
         ...,
         [ 5.0803e-04, -2

In [10]:
config = {}
config['row_mask_rate'] = 0
config['row_mask_pad'] = 0
config['rand_weight'] = 0
config['rand_base'] = 0
config['save_iter'] = 0
config['coeff_xy'] = 0 
config['nb_proj'] = 1
aug = None

In [11]:
net.eval()
# net.lstm.train()
# net.attention_layer.train()
batch_size = 1

# adv_perturbation: lrgb [224, 224, 2]  
input_size = 224
adv_perturbation = torch.zeros((input_size,input_size,2)).to(device)
adv_perturbation.requires_grad = True
expand_to_lrgb = torch.Tensor([[1, 0],[-0.2126/0.7152, -0.0722/0.7152],[0,1]]).T.to(device)

mask_range = input_size
mask_size = int(mask_range * config['row_mask_rate'])
t = torch.arange(0, mask_range, device=device).repeat(batch_size,1)
pad = config['row_mask_pad']

loss_sum = 0
save_iter = config['save_iter']
coeff_xy = config['coeff_xy']
nb_proj = config['nb_proj']
if coeff_xy >= 0:
    mseloss = torch.nn.MSELoss()

In [12]:

video_lrgb = rgb2lrgb(images[0])
net.zero_grad()

#           # expand perturbation
l_perturbation = torch.matmul(adv_perturbation, expand_to_lrgb).permute(2,0,1)

adv_aug = lrgb2rgb(video_lrgb+l_perturbation)
# # Random intensity
# weight = torch.rand(1).to(device)
# l_perturbation = (weight*config['rand_weight']+config['rand_base'])*l_perturbation

# # Row masking
# start_row = torch.randint(low=pad, high=mask_range-mask_size-pad, \
#                           size=(batch_size,1), device=device)
# mask = t.ge(start_row) & t.lt(start_row+mask_size)
# l_perturbation[mask.reshape(1,input_size,1).repeat(3,1,input_size)] = 0

# # add perturbation
# p_proj = project_perturbation(video_lrgb, l_perturbation, nb_proj)

# temp = video_lrgb + p_proj
# p_plus = torch.clamp(temp, 0, 1)
# idx_plus = p_plus != temp
# p_plus = p_plus - video_lrgb

# temp = video_lrgb - p_proj
# p_minus = torch.clamp(temp, 0, 1)
# idx_minus = p_minus != temp
# p_minus = video_lrgb - p_minus

# p_clipped = torch.where(p_plus.abs() > p_minus.abs(), p_minus, p_plus)
# p_clipped = torch.where(idx_plus | idx_minus, p_clipped, p_proj)
# l_perturbation = p_clipped

# # plus, minus, mixed
# adv_minus = lrgb2rgb(torch.clamp(video_lrgb - l_perturbation, 0, 1))
# adv_plus = lrgb2rgb(torch.clamp(video_lrgb + l_perturbation, 0, 1))

# # sample
# sequence_length = 40
# video_len = len(video_lrgb)
# sample_interval = np.random.randint(1, video_len // sequence_length + 1)
# start_i = np.random.randint(0, video_len - sample_interval * sequence_length + 1)

# adv_video = []
# for i in range(start_i, start_i + sequence_length*sample_interval, sample_interval):
#     if i % 2 == 0:
#         adv_video.append(adv_plus[i])
#     else:
#         adv_video.append(adv_minus[i])
# adv_video = torch.stack(adv_video)

# adv_video = lrgb2rgb(adv_video)
# if aug is not None:
#     adv_aug = aug(adv_video)
# else:
#     adv_aug = adv_video

In [13]:
outputs = net(adv_aug.unsqueeze(0))
print (outputs)

tensor([[0.0103, 0.0103, 0.0098, 0.0096, 0.0096, 0.0096, 0.0099, 0.0099, 0.0101,
         0.0097, 0.0095, 0.0100, 0.0103, 0.0101, 0.0096, 0.0097, 0.0098, 0.0102,
         0.0099, 0.0098, 0.0101, 0.0096, 0.0099, 0.0098, 0.0096, 0.0096, 0.0100,
         0.0095, 0.0102, 0.0100, 0.0101, 0.0099, 0.0105, 0.0098, 0.0095, 0.0100,
         0.0100, 0.0097, 0.0098, 0.0104, 0.0097, 0.0100, 0.0101, 0.0097, 0.0100,
         0.0098, 0.0102, 0.0098, 0.0100, 0.0096, 0.0100, 0.0099, 0.0099, 0.0101,
         0.0100, 0.0097, 0.0103, 0.0100, 0.0098, 0.0094, 0.0103, 0.0097, 0.0094,
         0.0096, 0.0099, 0.0097, 0.0103, 0.0098, 0.0096, 0.0099, 0.0101, 0.0095,
         0.0103, 0.0099, 0.0099, 0.0101, 0.0098, 0.0094, 0.0102, 0.0098, 0.0103,
         0.0100, 0.0099, 0.0104, 0.0099, 0.0098, 0.0095, 0.0096, 0.0100, 0.0098,
         0.0104, 0.0101, 0.0095, 0.0098, 0.0102, 0.0097, 0.0101, 0.0099, 0.0096,
         0.0104, 0.0099]], device='cuda:0', grad_fn=<SoftmaxBackward>)


In [14]:
loss_fn = torch.nn.CrossEntropyLoss()
loss_ce = loss_fn(outputs, labels+1)

In [15]:
loss_ce.backward()

RuntimeError: cudnn RNN backward can only be called in training mode

In [16]:
adv_perturbation.grad

In [ ]:

if coeff_xy >= 0:
    video_xyY = rgb2xyY(images)
    plus_xyY = rgb2xyY(adv_plus)
    minus_xyY = rgb2xyY(adv_minus)
    merged_xyY = (plus_xyY + minus_xyY) / 2
    loss_xy = mseloss(merged_xyY, rgb2xyY(video_xyY))

loss = loss_ce + coeff_xy * loss_xy
loss.backward()

grad = self._get_perturb(adv_perturbation.grad.detach())
if targeted >= 0:
    adv_perturbation.data = adv_perturbation.data - self.eps_iter * grad
else:
    adv_perturbation.data = adv_perturbation.data + self.eps_iter * grad
adv_perturbation.data = self._projection(adv_perturbation.data, 1)
adv_perturbation.grad.data.zero_()

print ("iter %d: l_ce, l_xy %.5f, loss %.5f" % \
       (i, loss_ce.item(), coeff_xy*loss_xy.item(), loss.item()), end='\r')
loss_sum += loss.item()


In [39]:
class LRGBLum0Attack():
    """
    LRGB attack in Luminance 0 plane
    """        
    def perturb(self, net, images, labels, config, device, targeted=False, verbose=False, aug=None):
        net.eval()
        net.lstm.train()
        net.attention_layer.train()
        batch_size = 1
        
        # adv_perturbation: lrgb [224, 224, 2]  
        input_size = 224
        adv_perturbation = torch.zeros((input_size,input_size,2)).to(device)
        adv_perturbation.requires_grad = True
        expand_to_lrgb = torch.Tensor([[1, 0],[-0.2126/0.7152, -0.0722/0.7152],[0,1]]).T.to(device)

        mask_range = input_size
        mask_size = int(mask_range * config['row_mask_rate'])
        t = torch.arange(0, mask_range, device=device).repeat(batch_size,1)
        pad = config['row_mask_pad']
        
        loss_sum = 0
        save_iter = config['save_iter']
        coeff_xy = config['coeff_xy']
        nb_proj = config['nb_proj']
        if coeff_xy >= 0:
            mseloss = torch.nn.MSELoss()
        for i in range(self.nb_iter):
            video_lrgb = rgb2lrgb(images[0])
            net.zero_grad()
            
#           # expand perturbation
            l_perturbation = torch.matmul(adv_perturbation,expand_to_lrgb).permute(2,0,1)
                
            # Random intensity
            weight = torch.rand(1).to(device)
            l_perturbation = (weight*config['rand_weight']+config['rand_base'])*l_perturbation
            
            # Row masking
            start_row = torch.randint(low=pad, high=mask_range-mask_size-pad, \
                                      size=(batch_size,1), device=device)
            mask = t.ge(start_row) & t.lt(start_row+mask_size)
            l_perturbation[mask.reshape(1,input_size,1).repeat(3,1,input_size)] = 0

            # add perturbation
            p_proj = project_perturbation(video_lrgb, l_perturbation, nb_proj)

            temp = video_lrgb + p_proj
            p_plus = torch.clamp(temp, 0, 1)
            idx_plus = p_plus != temp
            p_plus = p_plus - video_lrgb

            temp = video_lrgb - p_proj
            p_minus = torch.clamp(temp, 0, 1)
            idx_minus = p_minus != temp
            p_minus = video_lrgb - p_minus

            p_clipped = torch.where(p_plus.abs() > p_minus.abs(), p_minus, p_plus)
            p_clipped = torch.where(idx_plus | idx_minus, p_clipped, p_proj)
            l_perturbation = p_clipped
            
            # plus, minus, mixed
            adv_minus = lrgb2rgb(torch.clamp(video_lrgb - l_perturbation, 0, 1))
            adv_plus = lrgb2rgb(torch.clamp(video_lrgb + l_perturbation, 0, 1))

            # sample
            sequence_length = 40
            video_len = len(video_lrgb)
            sample_interval = np.random.randint(1, video_len // sequence_length + 1)
            start_i = np.random.randint(0, video_len - sample_interval * sequence_length + 1)

            adv_video = []
            for i in range(start_i, start_i + sequence_length*sample_interval, sample_interval):
                if i % 2 == 0:
                    adv_video.append(adv_plus[i])
                else:
                    adv_video.append(adv_minus[i])
            adv_video = torch.stack(adv_video)
            
            adv_video = lrgb2rgb(adv_video)
            if aug is not None:
                adv_aug = aug(adv_video)
            else:
                adv_aug = adv_video
                
            print (adv_aug.unsqueeze(0).shape)
            outputs = net(adv_aug.unsqueeze(0))
            
            loss_ce = self.loss_fn(outputs, labels)
            
            if coeff_xy >= 0:
                video_xyY = rgb2xyY(images)
                plus_xyY = rgb2xyY(adv_plus)
                minus_xyY = rgb2xyY(adv_minus)
                merged_xyY = (plus_xyY + minus_xyY) / 2
                loss_xy = mseloss(merged_xyY, rgb2xyY(video_xyY))
                
            loss = loss_ce + coeff_xy * loss_xy
            loss.backward()

            grad = self._get_perturb(adv_perturbation.grad.detach())
            if targeted >= 0:
                adv_perturbation.data = adv_perturbation.data - self.eps_iter * grad
            else:
                adv_perturbation.data = adv_perturbation.data + self.eps_iter * grad
            adv_perturbation.data = self._projection(adv_perturbation.data, 1)
            adv_perturbation.grad.data.zero_()
            
            print ("iter %d: l_ce, l_xy %.5f, loss %.5f" % \
                   (i, loss_ce.item(), coeff_xy*loss_xy.item(), loss.item()), end='\r')
            loss_sum += loss.item()
                
#             if (i+1) % save_iter == 0:
#                 file_name = '%s_%d.npy' % (config['file_name'], (i+1))
#                 np.save(config['file_dir'] + file_name, torch.matmul(adv_perturbation,expand_to_lrgb).permute(0,3,1,2).detach().cpu())
#                 print (datetime.datetime.now(), \
#                        'save the perturbation at %dth iterations' % (i+1))
                
#         file_name = '%s_%d.npy' % (config['file_name'], (i+1))
#         np.save(config['file_dir'] + file_name, torch.matmul(adv_perturbation,expand_to_lrgb).permute(0,3,1,2).detach().cpu())
#         print (datetime.datetime.now(), \
#                'save the perturbation at %dth iterations' % (i+1))
        return torch.matmul(adv_perturbation,expand_to_lrgb).permute(0,3,1,2).detach()

In [40]:

# Attack base class
class LinfAttack():
        
    def _rand_init(self, image):
        #     np.random.uniform(low=-self.eps, high=self.eps, size=image.shape)
        return image + torch.zeros_like(image).uniform_(-self.eps, self.eps)
    
    def _get_perturb(self, grad):
        return torch.sign(grad)
    
    def _projection(self, perturbation, budget):
        return torch.clamp(perturbation, -budget, budget)
    
class LinfLRGBLum0Attack(LRGBLum0Attack, LinfAttack):
    def __init__(self, loss_fn, eps, nb_iter, eps_iter, 
                 rand_init, clip_min, clip_max):
        self.loss_fn = loss_fn
        self.eps = eps
        self.nb_iter = nb_iter
        self.eps_iter = eps_iter
        self.rand_init = rand_init
        self.clip_min = clip_min
        self.clip_max = clip_max

In [41]:
attack = LinfLRGBLum0Attack(torch.nn.CrossEntropyLoss(), eps=0.1, nb_iter=5, eps_iter=0.1, rand_init=False, clip_min=0, clip_max=0)

In [42]:
config = {}
config['row_mask_rate'] = 0
config['row_mask_pad'] = 0
config['rand_weight'] = 0
config['rand_base'] = 0
config['save_iter'] = 0
config['coeff_xy'] = 0 
config['nb_proj'] = 1
attack.perturb(model, images, labels.to(device), config=config, device='cuda')

torch.Size([1, 40, 3, 224, 224])


/home/sangwooji/miniconda3/envs/colorfusion/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1, 164, 3, 224, 224])) that is different to the input size (torch.Size([164, 3, 224, 224])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling .backward() or autograd.grad() the first time.

In [38]:
model

ConvLSTM(
  (encoder): Encoder(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)

In [33]:
model.lstm.training

True

In [ ]:
# Old version
class LRGBLum0Attack():
    """
    LRGB attack in Luminance 0 plane
    """        
    def perturb(self, net, images, labels, config, device, targeted=False, verbose=False, aug=None):
        net.eval()
        batch_size = 1
        
        # adv_perturbation: lrgb [224, 224, 2]  
        input_size = 224
        adv_perturbation = torch.zeros((input_size,input_size,2)).to(device)
        adv_perturbation.requires_grad = True
        expand_to_lrgb = torch.Tensor([[1, 0],[-0.2126/0.7152, -0.0722/0.7152],[0,1]]).T.to(device)

        mask_range = input_size
        mask_size = int(mask_range * config['row_mask_rate'])
        t = torch.arange(0, mask_range, device=device).repeat(batch_size,1)
        pad = config['row_mask_pad']
        
        loss_sum = 0
        save_iter = config['save_iter']
        coeff_xy = config['coeff_xy']
        nb_proj = config['nb_proj']
        if coeff_xy >= 0:
            mseloss = torch.nn.MSELoss()
        for i in range(self.nb_iter):
            video = images.clone()[0]
            video_lrgb = rgb2lrgb(video)
            net.zero_grad()
            
#           # expand perturbation
            l_perturbation = torch.matmul(adv_perturbation,expand_to_lrgb).permute(2,0,1)
                
            # Random intensity
            weight = torch.rand(1).to(device)
            l_perturbation = (weight*config['rand_weight']+config['rand_base'])*l_perturbation
            
            # Row masking
            start_row = torch.randint(low=pad, high=mask_range-mask_size-pad, \
                                      size=(batch_size,1), device=device)
            mask = t.ge(start_row) & t.lt(start_row+mask_size)
            l_perturbation[mask.reshape(1,input_size,1).repeat(3,1,input_size)] = 0

            # add perturbation
            p_proj = project_perturbation(video_lrgb, l_perturbation, nb_proj)
            for i, image in enumerate(video_lrgb):
                if i % 2 == 0:
                    image = image + p_proj[i]
                else:
                    image = image - p_proj[i]
            clipped = torch.clamp(video_lrgb, 0, 1)

            # sample
            sequence_length = 40
            video_len = len(clipped)
            sample_interval = np.random.randint(1, video_len // sequence_length + 1)
            start_i = np.random.randint(0, video_len - sample_interval * sequence_length + 1)

            adv_video = []
            for i in range(start_i, start_i + sequence_length*sample_interval, sample_interval):
                adv_video.append(clipped[i])
            adv_video = torch.stack(adv_video)
            
            adv_video = lrgb2rgb(adv_video)
            if aug is not None:
                adv_aug = aug(adv_video)
            else:
                adv_aug = adv_video
            outputs = net(adv_aug)
            
            loss_ce = self.loss_fn(outputs, labels)
            
            if coeff_xy >= 0:
                video_compen = images.clone()[0]
                for i, image in enumerate(video_compen):
                    if i % 2 == 0:
                        image = image - p_proj[i]
                    else:
                        image = image + p_proj[i]
                video_compen = torch.clamp(video_compen, 0, 1)
                
                plus_xyY = rgb2xyY(adv_video)
                minus_xyY = lrgb2xyY(video_compen)
                merged_xyY = (plus_xyY + minus_xyY) / 2
                loss_xy = mseloss(merged_xyY, rgb2xyY(video))
                
            loss = loss_ce + coeff_xy * loss_xy
            loss.backward()

            grad = self._get_perturb(adv_perturbation.grad.detach())
            if targeted >= 0:
                adv_perturbation.data = adv_perturbation.data - self.eps_iter * grad
            else:
                adv_perturbation.data = adv_perturbation.data + self.eps_iter * grad
            adv_perturbation.data = self._projection(adv_perturbation.data, 1)
            adv_perturbation.grad.data.zero_()
            
            print ("iter %d: l_ce, l_xy %.5f, loss %.5f" % \
                   (i, loss_ce.item(), coeff_xy*loss_xy.item(), loss.item()), end='\r')
            loss_sum += loss.item()
                
#             if (i+1) % save_iter == 0:
#                 file_name = '%s_%d.npy' % (config['file_name'], (i+1))
#                 np.save(config['file_dir'] + file_name, torch.matmul(adv_perturbation,expand_to_lrgb).permute(0,3,1,2).detach().cpu())
#                 print (datetime.datetime.now(), \
#                        'save the perturbation at %dth iterations' % (i+1))
                
#         file_name = '%s_%d.npy' % (config['file_name'], (i+1))
#         np.save(config['file_dir'] + file_name, torch.matmul(adv_perturbation,expand_to_lrgb).permute(0,3,1,2).detach().cpu())
#         print (datetime.datetime.now(), \
#                'save the perturbation at %dth iterations' % (i+1))
        return torch.matmul(adv_perturbation,expand_to_lrgb).permute(0,3,1,2).detach()